In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

###SILVER LAYER SCRIPT

##DATA ACCESS USING APP

In [0]:
spark.conf.set("fs.azure.account.auth.type.awstoragedatalakedulla.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.awstoragedatalakedulla.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.awstoragedatalakedulla.dfs.core.windows.net", "129d0e5d-d106-4956-99a5-467c76dbd901")
spark.conf.set("fs.azure.account.oauth2.client.secret.awstoragedatalakedulla.dfs.core.windows.net", "*************************************")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.awstoragedatalakedulla.dfs.core.windows.net", "https://login.microsoftonline.com/c818cdfd-9306-4bd9-9f14-1bdc31a5d122/oauth2/token")

###DATA LOADING

Reading data

In [0]:
df_cal = spark.read.format('csv')\
        .option("header", True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Calendar')

In [0]:
df_cus = spark.read.format('csv')\
        .option("header", True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Customers')

In [0]:
df_procat = spark.read.format('csv')\
        .option("header", True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Product_Categories')

In [0]:
df_pro = spark.read.format('csv')\
        .option("header", True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Products')

In [0]:
df_sales = spark.read.format('csv')\
        .option("header", True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Sales*')

In [0]:
df_ter = spark.read.format('csv')\
        .option("header", True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Territories')

In [0]:
df_sub_cat = spark.read.format('csv')\
        .option("header", True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalakedulla.dfs.core.windows.net/Product_Subcategories')

In [0]:
df_ret = spark.read.format('csv')\
        .option("header", True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Returns')

###Transformations

In [0]:
df_cal= df_cal.withColumn('Month', month(col('Date')))\
    .withColumn('Year', year(col('Date')))

In [0]:
df_cal.display()

In [0]:
df_cal.write.format('parquet')\
    .mode('append')\
        .option('path','abfss://silver@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Calendar')\
            .save()

In [0]:
df_cus.withColumn('FullName',concat(col('Prefix'),lit(' '),col('FirstName'),lit(' '),col('LastName'))).display

<bound method apply_dataframe_display_patch.<locals>.df_display of DataFrame[CustomerKey: int, Prefix: string, FirstName: string, LastName: string, BirthDate: date, MaritalStatus: string, Gender: string, EmailAddress: string, AnnualIncome: string, TotalChildren: int, EducationLevel: string, Occupation: string, HomeOwner: string, FullName: string]>

In [0]:
df_cus = df_cus.withColumn('FullName',concat_ws(' ',col('Prefix'),col('FirstName'),col('LastName')))

In [0]:
df_cus.write.format('parquet')\
    .mode('append')\
        .option('path','abfss://silver@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Customers')\
            .save()

In [0]:
df_sub_cat.write.format('parquet')\
    .mode('append')\
        .option('path','abfss://silver@awstoragedatalakedulla.dfs.core.windows.net/Product_Subcategories')\
            .save()

In [0]:
df_pro.display()

ProductKey,ProductSubcategoryKey,ProductSKU,ProductName,ModelName,ProductDescription,ProductColor,ProductSize,ProductStyle,ProductCost,ProductPrice
214,31,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Red,0,0,13.0863,34.99
215,31,HL-U509,"Sport-100 Helmet, Black",Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Black,0,0,12.0278,33.6442
218,23,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,Combination of natural and synthetic fibers stays dry and provides just the right cushioning.,White,M,U,3.3963,9.5
219,23,SO-B909-L,"Mountain Bike Socks, L",Mountain Bike Socks,Combination of natural and synthetic fibers stays dry and provides just the right cushioning.,White,L,U,3.3963,9.5
220,31,HL-U509-B,"Sport-100 Helmet, Blue",Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Blue,0,0,12.0278,33.6442
223,19,CA-1098,AWC Logo Cap,Cycling Cap,Traditional style with a flip-up brim; one-size fits all.,Multi,0,U,5.7052,8.6442
226,21,LJ-0192-S,"Long-Sleeve Logo Jersey, S",Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,S,U,31.7244,48.0673
229,21,LJ-0192-M,"Long-Sleeve Logo Jersey, M",Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,M,U,31.7244,48.0673
232,21,LJ-0192-L,"Long-Sleeve Logo Jersey, L",Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,L,U,31.7244,48.0673
235,21,LJ-0192-X,"Long-Sleeve Logo Jersey, XL",Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,XL,U,31.7244,48.0673


Remove the first 2 letters from the ProductSKU splitting from the hyphen and doing same to the prdouct name splitting from the space

In [0]:
df_pro = df_pro.withColumn('ProductSKU',split(col('ProductSKU'),'-')[0])\
    .withColumn('ProductName',split(col('ProductName'),' ')[0])

In [0]:
df_pro.write.format('parquet')\
    .mode('append')\
        .option('path','abfss://silver@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Products')\
            .save()

In [0]:
df_ret.write.format('parquet')\
    .mode('append')\
        .option('path','abfss://silver@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Returns')\
            .save()

In [0]:
df_ter.write.format('parquet')\
    .mode('append')\
        .option('path','abfss://silver@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Territories')\
            .save()

In [0]:
df_sales = df_sales.withColumn('StockDate',to_timestamp('StockDate'))

In [0]:
df_sales = df_sales.withColumn('OrderNumber', regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_sales = df_sales.withColumn('Multiply',col('OrderLineItem')* col("OrderQuantity"))

In [0]:
df_sales.display()

In [0]:
df_sales.write.format('parquet')\
    .mode('append')\
        .option('path','abfss://silver@awstoragedatalakedulla.dfs.core.windows.net/AdventureWorks_Sales*')\
            .save()

###Sales Analysis

In [0]:
df_sales.groupby('OrderDate').agg(count('OrderNumber').alias('Total_Order')).display()

In [0]:
df_procat.display()

ProductCategoryKey,CategoryName
1,Bikes
2,Components
3,Clothing
4,Accessories


Databricks visualization. Run in Databricks to view.

In [0]:
df_ter.display()

SalesTerritoryKey,Region,Country,Continent
1,Northwest,United States,North America
2,Northeast,United States,North America
3,Central,United States,North America
4,Southwest,United States,North America
5,Southeast,United States,North America
6,Canada,Canada,North America
7,France,France,Europe
8,Germany,Germany,Europe
9,Australia,Australia,Pacific
10,United Kingdom,United Kingdom,Europe


Databricks visualization. Run in Databricks to view.